In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 01_data_cleaning — Indian Cities GDP & Productivity (2019–2024)

This notebook cleans and merges two raw CSVs into a single **master dataset** for EDA.

**Raw inputs (Google Drive):**
- `.../Raw Data/Economy_Productivity_SD_India.csv`
- `.../Raw Data/GDP_Sector_Income_2019_2023_30Cities.csv`

**Clean output (Google Drive):**
- `.../Cleaned Data/indian_cities_gdp_productivity.csv`

Tip: Paths are configured in the next cell to match your Drive layout.


In [2]:
import pandas as pd
from pathlib import Path

# Root folder in your Google Drive (adjust only if you move the project)
DRIVE_DIR = Path("/content/drive/MyDrive/GitHub/01 indian-cities-gdp-2019-2024")

# Paths to raw input files (these match what you shared)
RAW_FILE_PRODUCTIVITY = DRIVE_DIR / "Raw Data/Economy_Productivity_SD_India.csv"
RAW_FILE_GDP          = DRIVE_DIR / "Raw Data/GDP_Sector_Income_2019_2023_30Cities.csv"

# Path for cleaned output
OUTPUT_FILE = DRIVE_DIR / "Cleaned Data/indian_cities_gdp_productivity.csv"
OUTPUT_FILE.parent.mkdir(parents=True, exist_ok=True)

RAW_FILE_PRODUCTIVITY, RAW_FILE_GDP, OUTPUT_FILE


(PosixPath('/content/drive/MyDrive/GitHub/01 indian-cities-gdp-2019-2024/Raw Data/Economy_Productivity_SD_India.csv'),
 PosixPath('/content/drive/MyDrive/GitHub/01 indian-cities-gdp-2019-2024/Raw Data/GDP_Sector_Income_2019_2023_30Cities.csv'),
 PosixPath('/content/drive/MyDrive/GitHub/01 indian-cities-gdp-2019-2024/Cleaned Data/indian_cities_gdp_productivity.csv'))

In [3]:
df_prod = pd.read_csv(RAW_FILE_PRODUCTIVITY)
df_gdp  = pd.read_csv(RAW_FILE_GDP)

print("Productivity shape:", df_prod.shape)
print("GDP shape:", df_gdp.shape)

display(df_prod.head(3))
display(df_gdp.head(3))


Productivity shape: (180, 9)
GDP shape: (150, 7)


,City,Year,R&D Expenditure (% of GDP),"Patents per 100,000 Inhabitants",Unemployment Rate (%),Youth Unemployment Rate (%),SME Employment (%),Tourism Sector Employment (%),ICT Sector Employment (%)
0,Ahmedabad,2019,1.06,3.4,5.6,11.8,38.5,5.2,20.7
1,Ahmedabad,2020,1.93,2.1,6.6,10.4,19.5,4.2,6.8
2,Ahmedabad,2021,1.60,6.6,5.8,8.9,16.7,5.1,13.4


,City,Year,GDP (in billion $),Agriculture (%),Industry (%),Services (%),Technology (%)
0,Ahmedabad,2019,187.2,15.7,32.1,40.9,20.6
1,Ahmedabad,2020,211.5,11.6,37.8,49.3,19.6
2,Ahmedabad,2021,247.9,12.9,31.4,48.5,11.8


In [5]:
def clean_columns(df):
    df = df.copy()
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
        .str.replace("%", "pct")
        .str.replace("(", "", regex=False)
        .str.replace(")", "", regex=False)
        .str.replace("/", "_per_")
    )
    return df

df_prod = clean_columns(df_prod)
df_gdp  = clean_columns(df_gdp)

df_prod.columns, df_gdp.columns


(Index(['city', 'year', 'r&d_expenditure_pct_of_gdp',
        'patents_per_100,000_inhabitants', 'unemployment_rate_pct',
        'youth_unemployment_rate_pct', 'sme_employment_pct',
        'tourism_sector_employment_pct', 'ict_sector_employment_pct'],
       dtype='object'),
 Index(['city', 'year', 'gdp_in_billion_$', 'agriculture_pct', 'industry_pct',
        'services_pct', 'technology_pct'],
       dtype='object'))

In [6]:
def quick_report(df, name):
    print(f"=== {name} ===")
    print("Shape:", df.shape)
    print("\nDtypes:")
    print(df.dtypes)
    print("\nMissing values:")
    print(df.isna().sum())
    print("-"*60)

quick_report(df_prod, "Productivity (raw, cleaned cols)")
quick_report(df_gdp,  "GDP (raw, cleaned cols)")


=== Productivity (raw, cleaned cols) ===
Shape: (180, 9)

Dtypes:
city                                object
year                                 int64
r&d_expenditure_pct_of_gdp         float64
patents_per_100,000_inhabitants    float64
unemployment_rate_pct              float64
youth_unemployment_rate_pct        float64
sme_employment_pct                 float64
tourism_sector_employment_pct      float64
ict_sector_employment_pct          float64
dtype: object

Missing values:
city                               0
year                               0
r&d_expenditure_pct_of_gdp         0
patents_per_100,000_inhabitants    0
unemployment_rate_pct              0
youth_unemployment_rate_pct        0
sme_employment_pct                 0
tourism_sector_employment_pct      0
ict_sector_employment_pct          0
dtype: int64
------------------------------------------------------------
=== GDP (raw, cleaned cols) ===
Shape: (150, 7)

Dtypes:
city                 object
year                  in

In [7]:
# Ensure 'year' is integer where possible
for d in (df_prod, df_gdp):
    if d["year"].dtype.kind not in ("i",):
        d["year"] = pd.to_numeric(d["year"], errors="coerce").astype("Int64")

# Identify the GDP column after cleaning (e.g., 'gdp_in_billion_$')
gdp_like_cols = [c for c in df_gdp.columns if "gdp" in c]
assert len(gdp_like_cols) >= 1, "No GDP column found in GDP dataset."
gdp_col = gdp_like_cols[0]

# Force GDP to numeric
df_gdp[gdp_col] = pd.to_numeric(df_gdp[gdp_col], errors="coerce")

# Standardize city names
for d in (df_prod, df_gdp):
    d["city"] = d["city"].astype(str).str.strip().str.title()

# Drop exact duplicates by (city, year)
df_prod = df_prod.drop_duplicates(subset=["city","year"]).copy()
df_gdp  = df_gdp.drop_duplicates(subset=["city","year"]).copy()

quick_report(df_prod, "Productivity (post-clean)")
quick_report(df_gdp,  "GDP (post-clean)")
print("GDP column used:", gdp_col)


=== Productivity (post-clean) ===
Shape: (180, 9)

Dtypes:
city                                object
year                                 int64
r&d_expenditure_pct_of_gdp         float64
patents_per_100,000_inhabitants    float64
unemployment_rate_pct              float64
youth_unemployment_rate_pct        float64
sme_employment_pct                 float64
tourism_sector_employment_pct      float64
ict_sector_employment_pct          float64
dtype: object

Missing values:
city                               0
year                               0
r&d_expenditure_pct_of_gdp         0
patents_per_100,000_inhabitants    0
unemployment_rate_pct              0
youth_unemployment_rate_pct        0
sme_employment_pct                 0
tourism_sector_employment_pct      0
ict_sector_employment_pct          0
dtype: int64
------------------------------------------------------------
=== GDP (post-clean) ===
Shape: (150, 7)

Dtypes:
city                 object
year                  int64
gdp_in_bil

In [8]:
# Rename GDP column to a friendly name
df_gdp_renamed = df_gdp.rename(columns={gdp_col: "gdp_billion_usd"})

# Inner merge to keep overlapping city-year pairs
df_master = pd.merge(
    df_gdp_renamed, df_prod,
    on=["city", "year"],
    how="inner",
    validate="one_to_one"
)

print("Master shape:", df_master.shape)
display(df_master.head(5))


Master shape: (75, 14)


,city,year,gdp_billion_usd,agriculture_pct,industry_pct,services_pct,technology_pct,r&d_expenditure_pct_of_gdp,"patents_per_100,000_inhabitants",unemployment_rate_pct,youth_unemployment_rate_pct,sme_employment_pct,tourism_sector_employment_pct,ict_sector_employment_pct
0,Ahmedabad,2019,187.2,15.7,32.1,40.9,20.6,1.06,3.4,5.6,11.8,38.5,5.2,20.7
1,Ahmedabad,2020,211.5,11.6,37.8,49.3,19.6,1.93,2.1,6.6,10.4,19.5,4.2,6.8
2,Ahmedabad,2021,247.9,12.9,31.4,48.5,11.8,1.60,6.6,5.8,8.9,16.7,5.1,13.4
3,Ahmedabad,2022,71.8,5.3,36.7,45.6,31.8,1.40,6.3,6.2,10.7,33.5,7.5,22.6
4,Ahmedabad,2023,294.7,17.0,29.2,45.6,13.0,0.73,2.9,7.8,10.5,29.4,6.5,23.9


In [9]:
# Sort for pct_change
df_master = df_master.sort_values(["city","year"])

# YoY GDP growth by city (%)
df_master["gdp_yoy_pct"] = (
    df_master.groupby("city")["gdp_billion_usd"]
    .pct_change()
    .mul(100)
)

# If sector % columns exist, compute absolute sector GDP (billions USD)
for col in ["agriculture_pct", "industry_pct", "services_pct", "technology_pct"]:
    if col in df_master.columns:
        abs_col = col.replace("_pct", "_gdp_billion")
        df_master[abs_col] = df_master["gdp_billion_usd"] * (df_master[col] / 100.0)

display(df_master.head(8))


,city,year,gdp_billion_usd,agriculture_pct,industry_pct,services_pct,technology_pct,r&d_expenditure_pct_of_gdp,"patents_per_100,000_inhabitants",unemployment_rate_pct,youth_unemployment_rate_pct,sme_employment_pct,tourism_sector_employment_pct,ict_sector_employment_pct,gdp_yoy_pct,agriculture_gdp_billion,industry_gdp_billion,services_gdp_billion,technology_gdp_billion
0,Ahmedabad,2019,187.2,15.7,32.1,40.9,20.6,1.06,3.4,5.6,11.8,38.5,5.2,20.7,NaN,29.3904,60.0912,76.5648,38.5632
1,Ahmedabad,2020,211.5,11.6,37.8,49.3,19.6,1.93,2.1,6.6,10.4,19.5,4.2,6.8,12.980769,24.5340,79.9470,104.2695,41.4540
2,Ahmedabad,2021,247.9,12.9,31.4,48.5,11.8,1.60,6.6,5.8,8.9,16.7,5.1,13.4,17.210402,31.9791,77.8406,120.2315,29.2522
3,Ahmedabad,2022,71.8,5.3,36.7,45.6,31.8,1.40,6.3,6.2,10.7,33.5,7.5,22.6,-71.036708,3.8054,26.3506,32.7408,22.8324
4,Ahmedabad,2023,294.7,17.0,29.2,45.6,13.0,0.73,2.9,7.8,10.5,29.4,6.5,23.9,310.445682,50.0990,86.0524,134.3832,38.3110
60,Amritsar,2019,124.1,19.9,25.0,32.1,33.8,1.45,2.5,7.9,9.3,15.5,8.6,20.4,NaN,24.6959,31.0250,39.8361,41.9458
61,Amritsar,2020,108.4,15.3,21.2,44.6,32.0,1.30,3.9,7.4,8.5,17.6,6.5,21.9,-12.651088,16.5852,22.9808,48.3464,34.6880
62,Amritsar,2021,118.1,10.7,27.5,45.0,15.9,0.64,3.7,7.4,8.3,35.0,6.5,20.2,8.948339,12.6367,32.4775,53.1450,18.7779


In [10]:
df_master.to_csv(OUTPUT_FILE, index=False)
print("Saved:", OUTPUT_FILE)
print("Exists?", OUTPUT_FILE.exists())


Saved: /content/drive/MyDrive/GitHub/01 indian-cities-gdp-2019-2024/Cleaned Data/indian_cities_gdp_productivity.csv
Exists? True


## Summary

- Standardized column names and types  
- Cleaned city names and removed duplicates  
- Merged GDP + productivity datasets on **City, Year**  
- Added **YoY GDP growth** and absolute **sector GDP** (if sector % present)  
- Saved cleaned file to Google Drive: `Cleaned Data/indian_cities_gdp_productivity.csv`

**Next:** Open `02_eda.ipynb` to explore trends and build charts.
